Exercise 1.
Sentiment Analysis

• Using textblob, what is the probability that the sentiment in the
Burbank text is going to negative?

Exercise 2.
Sentiment Analysis

• Using the data from exercise 1 and textblob, what is the overall
sentiment and subjectivity?

Exercise 3.

 Key topic using ‘Word’ from textblob (very simple way to
determine the key topics) based on the Burbank text file.

• Import Word from textblob. Identify the key topics by using Word
from textblob.

Exercise 4.

Sentiment analysis with spaCy.

• Load the datasets ‘amazon_cells_labelled.txt’, ‘imdb_labelled.txt’,
‘yelp_labelled.txt’

• Create ‘combined_col’ by joining the tables such that
combined_col=[data_amazon, data_imdb, data_yelp]

• Check the structure of data_amazon

• Add headers for columns in each dataset: ‘Review’ and ‘Label’

• Create a “Company’ column to identify each company ‘Amazon’,
‘imdb’, and ‘yelp’

• Explore the structure of the new dataset called ‘comb_data’

• Use ‘comb_data.to_csv’ to create the ‘Sentiment_Analysis_Dataset’

• Print the columns

• Check for null values

• Import STOP_WORDS from spacy and stopwords from
spacy.lang.en.stop_words

• Build a list of stopwords for filtering

• Import string, define ‘punctuations’ and define a ‘parser’

• Tokenize the sentences

• Import ‘CountVectorize’, ‘TfidVectorizer’, ‘accuracy_score’,
‘TransformerMixin’, ‘Pipeline’, and ‘LinearSVC’

• Create a class 'predictors(TransformerMixin)'. Within the class, define 'transform', 'fit', and 'get_params'

• Create a basic function to clean the text

• Vectorize and use LinearSVC as a classifier

• Use TfidfVectorizer

• Split the ‘com_data’ dataset into a train and test (20%) set

• Create a pipeline to clean, tokenize, vectorize, and classify as
‘pipe_countvect’

• Fit the data

• Predict with the test dataset

• Prediction results as ‘1’ for positive reviews, and ‘0’ for negative
reviews

• Use print(sample, “Prediction➔”, pred)

• Determine the accuracy for the test dataset, X_test/sample
prediction, and train dataset

## Excersice 1

In [ ]:
import textblob
from textblob import TextBlob

with open('Burbank.txt', 'r') as f:
    text = f.read()

blob = TextBlob(text)
sentiment = blob.sentiment

if sentiment[0] < 0:
    print("Probability of negative sentiment:", abs(sentiment[0]))
else:
    print("Probability of negative sentiment: 0")

Probability of negative sentiment: 0


In [ ]:
# Printing teh sentiment and Subjectivity of the text
print("Overall sentiment:", sentiment[0])
print("Subjectivity:", sentiment[1])

Overall sentiment: 0.09869334480780263
Subjectivity: 0.3790877796901893


## Excersice 2

In [ ]:
pip install nltk

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from textblob import TextBlob

with open('Burbank.txt', 'r') as f:
    text = f.read()

blob = TextBlob(text)
words = blob.words


In [ ]:
# Importing FreqDist
from nltk.probability import FreqDist


In [ ]:
# Creating fdist for most commonly repeted words
fdist = FreqDist(words)
frequent_words = fdist.most_common(5)

In [ ]:
# Printing the Top 5 key topics
print("Top 5 Key Topics:")
for word, freq in frequent_words:
    print(word)

Top 5 Key Topics:
the
of
to
and
that


## Excersice 3

In [ ]:
# Importing the Required Libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.base import TransformerMixin

In [ ]:
# Loading the  datasets
amazon = pd.read_csv('amazon_cells_labelled.txt', sep='\t', header=None)
imdb = pd.read_csv('imdb_labelled.txt', sep='\t', header=None)
yelp = pd.read_csv('yelp_labelled.txt', sep='\t', header=None)


In [ ]:
# Combining datasets
combined_col = pd.concat([amazon, imdb, yelp], ignore_index=True)


In [ ]:
# Adding column headers
combined_col.columns = ['Review', 'Label']


In [ ]:
# Adding a new column called  company
combined_col['Company'] = ''
combined_col.loc[combined_col.index < amazon.shape[0], 'Company'] = 'Amazon'
combined_col.loc[(combined_col.index >= amazon.shape[0]) & (combined_col.index < (amazon.shape[0] + imdb.shape[0])), 'Company'] = 'IMDB'
combined_col.loc[combined_col.index >= (amazon.shape[0] + imdb.shape[0]), 'Company'] = 'Yelp'

comb_data = combined_col
comb_data.to_csv('Sentiment_Analysis_Dataset.csv', index=False)

In [ ]:
# Exploring the  data
print(comb_data.columns)
print(comb_data.isnull().sum())


Index(['Review', 'Label', 'Company'], dtype='object')
Review     0
Label      0
Company    0
dtype: int64


In [ ]:
# Impoting the  stopwords
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
stopwords = list(STOP_WORDS)


In [ ]:
# Building the  stopwords list
import string
punctuations = string.punctuation
parser = spacy.lang.en.English()

In [ ]:
# Tokenizing the Reveiw
nltk_tokens =combined_col['Review'].apply(lambda x: parser(x))


In [ ]:
# Vectorizers and model
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC

In [ ]:
# Custom transformer
class predictors(TransformerMixin):

    def transform(self, X, **transform_params):
        return [clean_text(text) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}


In [ ]:
# Clean text function
def clean_text(text):
    return text.strip().lower()

In [ ]:
# Create pipeline
pipe_countvect = Pipeline([("cleaner", predictors()),
                   ('vectorizer', CountVectorizer()),
                   ('classifier', LinearSVC())])

In [ ]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(comb_data['Review'], comb_data['Label'], test_size=0.2)


In [ ]:
# Fit pipeline
pipe_countvect.fit(X_train,y_train)

Pipeline(steps=[('cleaner', <__main__.predictors object at 0x7dc18e7d2c20>),
                ('vectorizer', CountVectorizer()),
                ('classifier', LinearSVC())])

In [ ]:
# Predict
pred = pipe_countvect.predict(X_test)


In [ ]:
# Print results
for sample, pred in list(zip(X_test, pred))[:3]:
    print(sample, "Prediction -->", pred)


However, my recent experience at this particular location was not so good. Prediction --> 0
WORTHWHILE. Prediction --> 1
Phone falls out easily. Prediction --> 0


In [ ]:

# Accuracy
print("Test accuracy:", accuracy_score(y_test, pred_class))
print("Train accuracy:", pipe_countvect.score(X_train, y_train))

Test accuracy: 0.8236363636363636
Train accuracy: 0.9986351228389445
